# CO2 forcing


From O'Connor et al. 2021:

- +1.89 W m-2

In [ ]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

import matplotlib.pyplot as pl
import pandas as pd
import pooch
from scipy.optimize import root

In [ ]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [ ]:
scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
species = ['CO2', 'CH4', 'N2O']
species, properties = read_properties(species=species, filename='../data/species_configs_properties_vanilla.csv')
properties['CO2']['input_mode'] = 'concentration'
properties['CH4']['input_mode'] = 'concentration'
properties['N2O']['input_mode'] = 'concentration'
properties

In [ ]:
rcmip_concentration_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-concentrations-annual-means-v5-1-0.csv",
    known_hash="md5:0d82c3c3cdd4dd632b2bb9449a5c315f",
)
conc_df = pd.read_csv(rcmip_concentration_file)

In [ ]:
co2 = conc_df.loc[(conc_df['Scenario']=='ssp245')&(conc_df['Variable']=='Atmospheric Concentrations|CO2')&(conc_df['Region']=='World'),'1850':'2015'].values.squeeze()
co2[-2] = co2[-1]
ch4 = conc_df.loc[(conc_df['Scenario']=='historical')&(conc_df['Variable']=='Atmospheric Concentrations|CH4')&(conc_df['Region']=='World'),'1850'].values[0]
n2o = conc_df.loc[(conc_df['Scenario']=='historical')&(conc_df['Variable']=='Atmospheric Concentrations|N2O')&(conc_df['Region']=='World'),'1850'].values[0]

In [ ]:
target_forcing = 1.89

def erf_rootfinder(x):
    f = FAIR(temperature_prescribed=True)
    f.define_time(1850, 2015, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(species, properties)
    f.allocate()
    f.fill_species_configs(filename='../data/species_configs_properties_vanilla.csv')
    
    # 1850 baselines
    f.species_configs['baseline_concentration'].loc[dict(specie='CO2')] = co2[0]
    f.species_configs['baseline_concentration'].loc[dict(specie='CH4')] = ch4
    f.species_configs['baseline_concentration'].loc[dict(specie='N2O')] = n2o
    
#    f.fill_species_configs()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    
    # fill emissions and concentrations
    f.concentration.loc[dict(specie='CO2')] = co2[:, None, None]
    f.concentration.loc[dict(specie='CH4')] = ch4
    f.concentration.loc[dict(specie='N2O')] = n2o

    f.temperature[:] = 0

    # this is the variable we are changing: CO2 scale factor
    f.species_configs['forcing_scale'].loc[dict(specie="CO2")] = x
    
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    initialise(f.cumulative_emissions, 0) 
    f.run(progress=False)
    return f.forcing[-1, 0, 0, 0] - target_forcing

In [ ]:
sol = root(erf_rootfinder, 1)

In [ ]:
sol.x[0]

In [ ]:
co2